# PyForge CLI - User Guide for Databricks Serverless

This notebook demonstrates how to use PyForge CLI to convert various file formats in Databricks Serverless environments.

## Table of Contents
1. [Installation and Setup](#installation)
2. [Install Sample Datasets](#sample-datasets)
3. [List Available Datasets](#list-datasets)
4. [DBF File Conversion](#dbf-conversion)
5. [Excel File Conversion](#excel-conversion)
6. [CSV File Conversion](#csv-conversion)
7. [XML File Conversion](#xml-conversion)
8. [Working with Converted Data](#working-with-data)
9. [Command Reference](#command-reference)
10. [Database File Conversion (.mdb/.accdb) - Special Instructions](#database-conversion)

## 1. Installation and Setup <a id='installation'></a>

**Important:** In Databricks Serverless, always include the PyPI index URL for proper dependency resolution.

In [ ]:
# Install PyForge CLI with explicit PyPI index URL (required for Databricks Serverless)
%pip install pyforge-cli==1.0.9 --no-cache-dir --quiet --index-url https://pypi.org/simple/ --trusted-host pypi.org

In [ ]:
# Restart Python kernel to ensure clean imports
dbutils.library.restartPython()

In [ ]:
# Verify installation
import subprocess
import sys
import os

# Check PyForge version
result = subprocess.run(['pyforge', '--version'], capture_output=True, text=True)
print(f"✅ PyForge CLI Version: {result.stdout.strip()}")

# Check Python version
print(f"✅ Python Version: {sys.version}")

# Check PyForge location
which_result = subprocess.run(['which', 'pyforge'], capture_output=True, text=True)
print(f"✅ PyForge Location: {which_result.stdout.strip()}")

## 2. Install Sample Datasets <a id='sample-datasets'></a>

PyForge CLI includes a curated collection of sample datasets for testing all supported formats.

In [ ]:
# Define Unity Catalog volume path for sample datasets
volume_path = "dbfs:/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets"
local_volume_path = volume_path.replace('dbfs:', '/dbfs')

In [ ]:
%%sh
# Install sample datasets using shell command
pyforge install sample-datasets --output /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets --verbose

## 3. List Available Sample Datasets <a id='list-datasets'></a>

Let's explore what sample datasets are available for conversion.

In [ ]:
# List all files in the sample datasets directory
import pandas as pd

try:
    files = dbutils.fs.ls(volume_path)
    
    # Organize files by type
    file_data = []
    for file_info in files:
        file_name = file_info.name
        file_path = file_info.path
        file_size = file_info.size / (1024 * 1024)  # Convert to MB
        
        # Determine file type
        if file_name.endswith('.mdb'):
            file_type = 'Access Database (Legacy)'
        elif file_name.endswith('.accdb'):
            file_type = 'Access Database (Modern)'
        elif file_name.endswith('.dbf'):
            file_type = 'dBASE Database'
        elif file_name.endswith('.xlsx'):
            file_type = 'Excel Spreadsheet'
        elif file_name.endswith('.csv'):
            file_type = 'CSV File'
        elif file_name.endswith('.xml'):
            file_type = 'XML Document'
        elif file_name.endswith('.pdf'):
            file_type = 'PDF Document'
        else:
            file_type = 'Other'
        
        if not file_info.isDir():
            file_data.append({
                'File Name': file_name,
                'Type': file_type,
                'Size (MB)': round(file_size, 2),
                'Path': file_path
            })
    
    # Create DataFrame and display
    df_files = pd.DataFrame(file_data)
    df_files = df_files.sort_values(['Type', 'File Name'])
    
    print("📁 Available Sample Datasets:\n")
    display(df_files)
    
    # Summary by type
    print("\n📊 Summary by File Type:")
    summary = df_files.groupby('Type').agg({
        'File Name': 'count',
        'Size (MB)': 'sum'
    }).rename(columns={'File Name': 'Count', 'Size (MB)': 'Total Size (MB)'})
    display(summary)
    
except Exception as e:
    print(f"Error listing files: {e}")
    print("Please ensure the sample datasets were installed successfully.")

## 4. DBF File Conversion <a id='dbf-conversion'></a>

Convert dBASE database files with automatic encoding detection.

In [ ]:
%%sh
# Convert DBF file using shell command
echo "🔄 Converting customer_data.dbf..."
pyforge convert /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/customer_data.dbf \
    /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/converted/customer_data.parquet \
    --encoding cp1252 \
    --verbose

In [ ]:
# Read and display the converted DBF data
output_file = f"{volume_path}/converted/customer_data.parquet"
df_dbf = pd.read_parquet(output_file.replace('dbfs:/', '/dbfs/'))
print(f"📊 DBF Data: {len(df_dbf)} records, {len(df_dbf.columns)} columns")
print(f"Columns: {', '.join(df_dbf.columns)}")
display(df_dbf.head(5))

## 5. Excel File Conversion <a id='excel-conversion'></a>

Convert Excel files with support for multiple sheets.

In [ ]:
%%sh
# Convert Excel file with sheet combination
echo "🔄 Converting financial_report.xlsx..."
pyforge convert /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/financial_report.xlsx \
    /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/converted/financial/ \
    --combine \
    --compression gzip \
    --verbose

In [ ]:
# List and analyze converted Excel files
output_dir = f"{volume_path}/converted/financial/"
parquet_files = [f for f in dbutils.fs.ls(output_dir) if f.name.endswith('.parquet')]

for file in parquet_files:
    df = pd.read_parquet(file.path.replace('dbfs:/', '/dbfs/'))
    print(f"\n📊 {file.name}:")
    print(f"   - Rows: {len(df):,}")
    print(f"   - Columns: {len(df.columns)}")
    print(f"   - Column Names: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
    display(df.head(3))

## 6. CSV File Conversion <a id='csv-conversion'></a>

Convert CSV files with automatic delimiter and encoding detection.

In [ ]:
%%sh
# Convert CSV file
echo "🔄 Converting sales_data.csv..."
pyforge convert /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/sales_data.csv \
    /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/converted/sales_data.parquet \
    --compression snappy \
    --verbose

In [ ]:
# Read and analyze the converted CSV data
output_file = f"{volume_path}/converted/sales_data.parquet"
df_csv = pd.read_parquet(output_file.replace('dbfs:/', '/dbfs/'))
print(f"📊 Sales Data Analysis:")
print(f"   - Total Records: {len(df_csv):,}")
print(f"   - Columns: {', '.join(df_csv.columns)}")
print(f"   - Data Types:\n{df_csv.dtypes}")
display(df_csv.head())

## 7. XML File Conversion <a id='xml-conversion'></a>

Convert XML files with intelligent structure flattening.

In [ ]:
%%sh
# Convert XML file with moderate flattening
echo "🔄 Converting books_catalog.xml..."
pyforge convert /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/books_catalog.xml \
    /dbfs/Volumes/cortex_dev_catalog/sandbox_testing/pyforge_sample_datasets/converted/books_catalog.parquet \
    --flatten-strategy moderate \
    --array-handling expand \
    --compression gzip \
    --verbose

In [ ]:
# Read and display the flattened XML data
output_file = f"{volume_path}/converted/books_catalog.parquet"
df_xml = pd.read_parquet(output_file.replace('dbfs:/', '/dbfs/'))
print(f"📚 Books Catalog:")
print(f"   - Total Books: {len(df_xml)}")
print(f"   - Columns: {', '.join(df_xml.columns)}")
display(df_xml.head())

## 8. Working with Converted Data <a id='working-with-data'></a>

Load and analyze the converted Parquet files using both Pandas and Spark.

### Using Pandas for Small to Medium Datasets

In [ ]:
# Example: Analyze converted CSV data
sales_path = f"/dbfs/{volume_path.replace('dbfs:/', '')}/converted/sales_data.parquet"
sales_pd = pd.read_parquet(sales_path)

print("📊 Pandas Analysis - Sales Data")
print(f"Total records: {len(sales_pd)}")
print(f"Columns: {', '.join(sales_pd.columns)}")
print("\nData summary:")
display(sales_pd.describe())

### Using Spark for Large-Scale Processing

In [ ]:
# Load converted files as Spark DataFrames
sales_spark = spark.read.parquet(f"{volume_path}/converted/sales_data.parquet")

# Register as temporary view for SQL queries
sales_spark.createOrReplaceTempView("sales")

print("✅ Spark DataFrame created and registered as view")
print(f"   - sales: {sales_spark.count()} rows")

# Run SQL analytics
result = spark.sql("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM sales
""")

display(result)

## 9. PyForge CLI Command Reference <a id='command-reference'></a>

### 📋 Basic Commands:

```bash
# Check version
pyforge --version

# Get help
pyforge --help
pyforge convert --help

# Install sample datasets
pyforge install sample-datasets --output /path/to/output
```

### 🔄 Conversion Commands:

```bash
# Basic conversion
pyforge convert input.csv output.parquet

# With compression
pyforge convert input.csv output.parquet --compression gzip

# Excel with sheet combination
pyforge convert data.xlsx output/ --combine

# XML with flattening options
pyforge convert data.xml output.parquet --flatten-strategy moderate --array-handling expand

# DBF with encoding
pyforge convert data.dbf output.parquet --encoding cp1252
```

### ⚙️ Common Options:

- `--compression`: gzip, snappy, brotli, lz4 (default: snappy)
- `--verbose`: Show detailed conversion progress
- `--encoding`: Specify character encoding (auto-detected by default)
- `--combine`: Combine multiple sheets into one file (Excel only)
- `--flatten-strategy`: conservative, moderate, aggressive (XML only)
- `--array-handling`: expand, preserve (XML only)

## 10. Database File Conversion (.mdb/.accdb) - Special Instructions <a id='database-conversion'></a>

### ⚠️ Important: MDB/ACCDB Files Require Subprocess

Due to Java SDK dependencies, MDB/ACCDB files **MUST** use subprocess commands instead of `%sh` magic commands.

- ✅ **MDB/ACCDB:** `subprocess.run(['pyforge', 'convert', 'file.mdb'])`
- ❌ **Do NOT use:** `%sh pyforge convert file.mdb`

### Convert MDB File (Access 2000-2003)

In [ ]:
# Convert Northwind.mdb database
mdb_file = f"{volume_path}/Northwind.mdb"
output_dir = f"{volume_path}/converted/northwind/"

print("🔄 Converting Northwind.mdb database...")
print(f"Input: {mdb_file}")
print(f"Output: {output_dir}")
print("\n" + "="*60 + "\n")

# Use subprocess for MDB files (REQUIRED due to Java SDK dependencies)
result = subprocess.run([
    'pyforge', 'convert',
    mdb_file,
    output_dir,
    '--compression', 'gzip',
    '--verbose'
], capture_output=True, text=True)

print("Conversion Output:")
print(result.stdout)

if result.returncode == 0:
    print("\n✅ Conversion successful!")
    
    # List converted tables
    converted_files = dbutils.fs.ls(output_dir)
    tables = [f.name for f in converted_files if f.name.endswith('.parquet')]
    
    print(f"\n📊 Converted {len(tables)} tables:")
    for table in sorted(tables):
        print(f"  - {table}")
else:
    print(f"\n❌ Conversion failed!")
    print(f"Error: {result.stderr}")

### Convert ACCDB File (Access 2007+)

In [ ]:
# Convert a modern Access database with specific tables
accdb_file = f"{volume_path}/AdventureWorks.accdb"
output_dir = f"{volume_path}/converted/adventureworks/"

print("🔄 Converting AdventureWorks.accdb database...")
print("Converting only specific tables: Customers, Orders, Products")
print(f"Input: {accdb_file}")
print(f"Output: {output_dir}")
print("\n" + "="*60 + "\n")

# Use subprocess for ACCDB files (REQUIRED due to Java SDK dependencies)
result = subprocess.run([
    'pyforge', 'convert',
    accdb_file,
    output_dir,
    '--tables', 'Customers,Orders,Products',  # Convert only these tables
    '--compression', 'snappy',
    '--verbose'
], capture_output=True, text=True)

print("Conversion Output:")
print(result.stdout)

if result.returncode == 0:
    print("\n✅ Conversion successful!")
    
    # Read and display sample data from Customers table
    customers_path = f"{output_dir}Customers.parquet"
    df_customers = pd.read_parquet(customers_path.replace('dbfs:/', '/dbfs/'))
    print(f"\n📊 Customers Table: {len(df_customers)} records")
    display(df_customers.head(5))
else:
    print(f"\n❌ Conversion failed: {result.stderr}")

### MDB/ACCDB Command Reference

```python
# Basic MDB conversion
subprocess.run(['pyforge', 'convert', 'database.mdb', 'output_dir/'])

# Convert specific tables
subprocess.run(['pyforge', 'convert', 'database.accdb', 'output/', '--tables', 'Table1,Table2'])

# With compression
subprocess.run(['pyforge', 'convert', 'database.mdb', 'output/', '--compression', 'gzip'])
```

In [ ]:
# Clean up example (optional)
# Uncomment to remove converted files
# dbutils.fs.rm(f"{volume_path}/converted/", recurse=True)
print("🎉 Tutorial complete! Your converted files are available in the volume.")

## Important Notes for Databricks Serverless

1. **Use %sh commands** for CSV, XML, Excel, DBF file conversions
2. **Use subprocess.run()** ONLY for MDB/ACCDB files due to Java SDK dependencies
3. **Include PyPI index URL** when installing: `--index-url https://pypi.org/simple/`
4. **Use dbfs:// prefix** for Unity Catalog volume paths in Python code
5. **Use /dbfs/ prefix** for shell commands
6. **Restart Python kernel** after installation with `dbutils.library.restartPython()`